# Introduction

This document contains information regarding the evaluation of the classification model using 3D images. That is, it shows how accurately the 3D image model can classify autism and nonautism test images.

In [ ]:
import os
import math
import glob

from keras.layers import Input,Dense,Flatten,Dropout,Conv3D,MaxPooling3D,Activation
from keras.models import Model,Sequential,load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras import regularizers
from keras import activations

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from skimage.transform import resize
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

import numpy as np
import scipy.misc

from PIL import Image
import nibabel as nib #reading MR images

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap


## Loading test images

In [ ]:
def count_nii_files_center(center_folder):
    nii_files = glob.glob(os.path.join(center_folder, '**', '*.nii'), recursive=True)
    return len(nii_files)

main_folder = 'Autism_TestData'  

autism_folder = os.path.join(main_folder, 'Autism')
nonautism_folder = os.path.join(main_folder, 'NonAutism')

autism_centers = [f.path for f in os.scandir(autism_folder) if f.is_dir()]
nonautism_centers = [f.path for f in os.scandir(nonautism_folder) if f.is_dir()]

autism_total_count = 0
nonautism_total_count = 0

for center_folder in autism_centers:
    nii_count = count_nii_files_center(center_folder)
    autism_total_count += nii_count
    center_name = os.path.basename(center_folder)
    print(f"Autism - Recruiting Center: {center_name}, Number of .nii files: {nii_count}")

for center_folder in nonautism_centers:
    nii_count = count_nii_files_center(center_folder)
    nonautism_total_count += nii_count
    center_name = os.path.basename(center_folder)
    print(f"Nonautism - Recruiting Center: {center_name}, Number of .nii files: {nii_count}")

print(f"Total number of .nii files in Autism folder: {autism_total_count}")
print(f"Total number of .nii files in NonAutism folder: {nonautism_total_count}")


## Data Preparation

Similar to the training phase I extract some slices from each axis so that the dimensions of the images are consistent and I can run the model.
The test dataset contains 52 images.
- **Autism**: 20 images
- **Nonautism**: 32 images

In [ ]:
def preprocess_image(image_path):
    img = nib.load(image_path).get_fdata()
    
    # Extract middle slices along y-axis
    img = img[30:110, 60:126, 60:126]
        
    # Z-score Standardization
    mean = img.mean()
    std = img.std()
    img = (img - mean) / std
    
    return img

def load_images_and_labels(data_folder, label):
    images = []
    labels = []
    
    # Get the list of .nii files
    image_paths = glob.glob(os.path.join(data_folder, '*.nii'))
    
    for image_path in image_paths:
        img = preprocess_image(image_path)
        images.append(img)
        labels.append(label)
    
    return images, labels

# Prepare the data
data_root = 'TestDF'  # Replace with your actual root directory
autism_folder = os.path.join(data_root, 'Autism')
nonautism_folder = os.path.join(data_root, 'NonAutism')

autism_images, autism_labels = load_images_and_labels(autism_folder, 1)
print("Autism Images Length:", len(autism_images))


# Print dimensions of each autism image
for i, img in enumerate(autism_images):
    print(f"Autism Image {i + 1} Dimensions: {img.shape}")

# Load and preprocess nonautism images
nonautism_images, nonautism_labels = load_images_and_labels(nonautism_folder, 0)
print("Nonautism Images Length:", len(nonautism_images))

# Print dimensions of each nonautism image
for i, img in enumerate(nonautism_images):
    print(f"Nonautism Image {i + 1} Dimensions: {img.shape}")


# Combine the data and labels
images = np.concatenate((autism_images, nonautism_images), axis=0)
labels = autism_labels + nonautism_labels

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)
labels = to_categorical(labels)  # Convert labels to one-hot encoded format


print("Images Shape:", images.shape)
print("Labels Shape:", labels.shape)



## Testing model's classification accuracy

In [ ]:
def preprocess_image(image_path):
    img = nib.load(image_path).get_fdata()
    
    # Extract middle slices along y-axis
    img = img[30:110, 60:126, 60:126]
        
    # Z-score Standardization
    mean = img.mean()
    std = img.std()
    img = (img - mean) / std
    
    return img

# Define the load_images_and_labels function
def load_images_and_labels(data_folder, label):
    images = []
    labels = []
    
    # Get the list of .nii files
    nii_files = glob.glob(os.path.join(data_folder, '*.nii'))
    
    for image_path in nii_files:
        img = preprocess_image(image_path)
        images.append(img)
        labels.append(label)
    
    return images, labels

# Load the trained model
model = load_model('Autism_model.h5')  # Replace with the actual path to your model

# Load and preprocess the images
data_root = 'TestDF'  # Replace with your actual root directory
autism_folder = os.path.join(data_root, 'Autism')
nonautism_folder = os.path.join(data_root, 'NonAutism')

autism_images, autism_labels = load_images_and_labels(autism_folder, 1)
nonautism_images, nonautism_labels = load_images_and_labels(nonautism_folder, 0)

# Combine the data and labels
images = np.concatenate((autism_images, nonautism_images), axis=0)
labels = np.concatenate((autism_labels, nonautism_labels), axis=0)

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)
labels = to_categorical(labels)  # Convert labels to one-hot encoded format

# Make predictions using the model
predictions = model.predict(images)

# Convert predictions to class labels (0 or 1)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == labels.argmax(axis=1)) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Print actual and predicted classifications
for i in range(len(images)):
    actual_classification = 'Autism' if labels[i][1] == 1 else 'NonAutism'
    predicted_classification = 'Autism' if predicted_labels[i] == 1 else 'NonAutism'
    
    print(f"Image {i+1} - Actual: {actual_classification} - Predicted: {predicted_classification}")


## Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(labels.argmax(axis=1), predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)


In [ ]:
# Create a heatmap
plt.figure(figsize=(9, 7))
sns.set(font_scale=1.5)

sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="OrRd", xticklabels=["NonAutism", "Autism"], yticklabels=["NonAutism", "Autism"])
plt.xlabel("Predicted",  fontsize=14, fontweight='bold')
plt.ylabel("Actual",  fontsize=14, fontweight='bold')
plt.title("Confusion Matrix",  fontsize=14, fontweight='bold')
plt.savefig("Autism_Confusion Matrix.png", dpi = 300)
plt.show()


## Results

In [ ]:
recall = recall_score(labels.argmax(axis=1), predicted_labels)
precision = precision_score(labels.argmax(axis=1), predicted_labels)
f1 = f1_score(labels.argmax(axis=1), predicted_labels)

print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"F1-Score: {f1:.2f}")


**Accuracy: 63.46%**
    
**Recall: 10%**
    
**Precision: 67%**


## Limitations

There are a number of limitations which might have contributed to the not high accuracy of the classification model:
 1. **Computational resources and hardware capacity**:
 Running a training model with 3D images require high speed computers with big capacity so that it contributes to the processing time.

 2. **Sample size**:
 As mentioned in the other document, due to the above reasons, I had to minimize the number of images used for the training model. However, more sample size can provide more amount of information and therefore can contribute to better classification and accuracy. Thus, more sample size can be used if there is more computational power.

 3. **Model complexity**:
 The majority of the models and features are designed for 2D image classification which makes the 3D model training model complex.
